In [37]:
from torch._inductor.select_algorithm import get_num_workers

from preprocess_data import DataPreprocessor
# Imports
import optuna
import numpy as np
import helpers


data_preprocessor = DataPreprocessor()

X_train, X_test, Y_train, Y_test = data_preprocessor.preprocess_data(lot_frontage_threshold=13)

to_remove = ['ohe__Utilities_NoSeWa', 'ohe__Neighborhood_Blueste', 'ohe__Condition1_RRNe', 'ohe__Condition2_PosA', 'ohe__Condition2_RRAe', 'ohe__Condition2_RRAn', 'ohe__Condition2_RRNn', 'ohe__RoofMatl_Membran', 'ohe__RoofMatl_Metal', 'ohe__RoofMatl_Roll', 'ohe__Exterior1st_AsphShn', 'ohe__Exterior1st_CBlock', 'ohe__Exterior1st_ImStucc', 'ohe__Exterior1st_Stone', 'ohe__Exterior2nd_CBlock', 'ohe__Exterior2nd_Other', 'ohe__Electrical_Mix', 'ohe__MiscFeature_TenC'] #Removed based on <= 1 total non-zero appearances

X_train = X_train.drop(to_remove, axis=1)
X_test = X_test.drop(to_remove, axis=1)

Calculated Global Median Ratio: 0.7235 (from 951 samples)
Calculating for group level: 3way (['MSZoning', 'BldgType', 'LotShape'])
 -> Found 39 groups for 3way
Calculating for group level: 2way_ZS (['MSZoning', 'LotShape'])
 -> Found 16 groups for 2way_ZS
Calculating for group level: 2way_ZB (['MSZoning', 'BldgType'])
 -> Found 19 groups for 2way_ZB
Calculating for group level: 2way_BS (['BldgType', 'LotShape'])
 -> Found 14 groups for 2way_BS
Calculating for group level: 1way_Z (['MSZoning'])
 -> Found 5 groups for 1way_Z
Calculating for group level: 1way_B (['BldgType'])
 -> Found 5 groups for 1way_B
Calculating for group level: 1way_S (['LotShape'])
 -> Found 4 groups for 1way_S


In [38]:
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

log_then_minmax = Pipeline([
    ('log_transform', FunctionTransformer(np.log1p)), # Example log transform
    ('min_max_scaler', MinMaxScaler())
])

ordinal_then_minmax_pipeline = Pipeline([
    ('ordinal_encode', OrdinalEncoder(
        categories=helpers.get_ordinal_cats_ordered(), # Make sure this returns the correct list of lists for categories
        handle_unknown='use_encoded_value',
        unknown_value=-1 # Or np.nan, but -1 works fine with MinMaxScaler
    )),
    ('minmax_scale_ordinal', MinMaxScaler()) # Scale the 0,1,2... output of OrdinalEncoder to [0,1]
])

model_pipeline = ColumnTransformer(
        transformers=[
            ('log_num', log_then_minmax, helpers.get_log_minmax_cols()),
            ('ord', ordinal_then_minmax_pipeline, helpers.get_categorical_cols_ordinal()),
            ('num', MinMaxScaler(), helpers.get_minmax_cols())
        ],
        remainder='passthrough',
        sparse_threshold=1
    )

In [39]:
from torch.utils.data import DataLoader, TensorDataset
# Deep learning model:
import torch
from torch import nn

model_pipeline.fit(X_train, Y_train)
X_train = model_pipeline.transform(X_train)
X_test = model_pipeline.transform(X_test)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# float64 acceptable for EDA, float32 preferred for training.
X_train = torch.tensor(X_train.values, device=device, dtype=torch.float32)
X_test = torch.tensor(X_test.values, device=device, dtype=torch.float32)
Y_train = torch.tensor(Y_train.values, device=device, dtype=torch.float32)
Y_test = torch.tensor(Y_test.values, device=device, dtype=torch.float32)


model = nn.Sequential(
    nn.Linear(X_train.shape[1], 302),
    nn.ELU(),
    nn.Linear(302, 75),
    nn.LeakyReLU(),
    nn.Linear(75, 18),
    nn.LeakyReLU(),
    nn.Linear(18, 1)
)

try:
    train_dataset = TensorDataset(X_train, Y_train)
    val_dataset = TensorDataset(X_test, Y_test)
    print(f"Train dataset length: {len(train_dataset)}")
    print(f"Validation dataset length: {len(val_dataset)}")
except Exception as e:
    print(f"Error creating TensorDataset: {e}")
    # Likely length mismatch between X and y tensors if error here

batch_size = 64 # Batch size lowered, due to sample size being less than ideal.
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True) # Shuffle training data
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False) # No need to shuffle validation

loss_func = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.09078721496248407,
                             betas=(0.8149819299051041, 0.9976723251615085),
                             eps=2.3572355850494554e-08)

model.to(device)

Train dataset length: 1168
Validation dataset length: 292


Sequential(
  (0): Linear(in_features=242, out_features=302, bias=True)
  (1): ELU(alpha=1.0)
  (2): Linear(in_features=302, out_features=75, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
  (4): Linear(in_features=75, out_features=18, bias=True)
  (5): LeakyReLU(negative_slope=0.01)
  (6): Linear(in_features=18, out_features=1, bias=True)
)

num_epochs = 155 # Hyperparameter: How many times to iterate over the dataset

def get_choice(activation_choice_name):
    """Helper function to get an activation function instance."""
    if activation_choice_name == 'sigmoid':
        return nn.Sigmoid()
    elif activation_choice_name == 'tanh':
        return nn.Tanh()
    elif activation_choice_name == 'leaky_relu':
        return nn.LeakyReLU()
    elif activation_choice_name == 'elu':
        return nn.ELU()
    elif activation_choice_name == 'silu': # Swish
        return nn.SiLU()
    else: # Default to ReLU
        return nn.ReLU()

def objective(trial):
    adam_lr = trial.suggest_float("adam_lr", 0.09, 0.1, log=True)
    adam_beta1 = trial.suggest_float("adam_beta1", 0.8, 0.95)
    adam_beta2 = trial.suggest_float("adam_beta2", 0.9, 0.999)
    adam_epsilon = trial.suggest_float("adam_epsilon", 1e-9, 1e-7, log=True)
    activation_choice_0 = trial.suggest_categorical('activation_0', ['relu', 'leaky_relu', 'elu', 'silu', 'tanh'])
    activation_choice_1 = trial.suggest_categorical('activation_1', ['relu', 'leaky_relu', 'elu', 'silu', 'tanh'])
    activation_choice_2 = trial.suggest_categorical('activation_2', ['relu', 'leaky_relu', 'elu', 'silu', 'tanh'])
    
    activation_0 = get_choice(activation_choice_0)
    activation_1 = get_choice(activation_choice_1)
    activation_2 = get_choice(activation_choice_2)
    
    input_features = X_train.shape[1]
    
    neurons_h1 = trial.suggest_categorical('neurons_h1', [
        max(32, int(input_features * 0.5)), 
        max(32, int(input_features * 0.75)),
        max(32, int(input_features * 1.0)),
        max(32, int(input_features * 1.25)),
        max(32, int(input_features * 1.5))
    ])
    
    # Neurons for Hidden Layer 2 (H2) - descending from H1
    # Using suggest_int with dynamic high bound based on neurons_h1
    # Ensure low is less than or equal to high.
    low_h2 = max(16, int(neurons_h1 * 0.25))
    high_h2 = neurons_h1
    if low_h2 > high_h2: # Ensure low is not greater than high
        low_h2 = high_h2 
    step_h2 = max(1, (high_h2 - low_h2) // 4) if high_h2 > low_h2 else 1 # Ensure step is at least 1
    if high_h2 == low_h2: # If low and high are same, suggest_int might error with step > 1
        neurons_h2 = high_h2
    else:
        neurons_h2 = trial.suggest_int('neurons_h2', low=low_h2, high=high_h2, step=max(1, step_h2 // 4 * 4 if step_h2 > 4 else step_h2)) # Ensure step is reasonable

    # Neurons for Hidden Layer 3 (H3) - descending from H2
    # Using suggest_int with dynamic high bound based on neurons_h2
    # Ensure low is less than or equal to high.
    low_h3 = max(8, int(neurons_h2 * 0.25))
    high_h3 = neurons_h2
    if low_h3 > high_h3: # Ensure low is not greater than high
        low_h3 = high_h3
    step_h3 = max(1, (high_h3 - low_h3) // 4) if high_h3 > low_h3 else 1 # Ensure step is at least 1
    if high_h3 == low_h3:
        neurons_h3 = high_h3
    else:
        neurons_h3 = trial.suggest_int('neurons_h3', low=low_h3, high=high_h3, step=max(1, step_h3 // 2 * 2 if step_h3 > 2 else step_h3))
    
    model = nn.Sequential(
        nn.Linear(input_features, neurons_h1),
        activation_0,
        nn.Linear(neurons_h1, neurons_h2),
        activation_1,
        nn.Linear(neurons_h2, neurons_h3),
        activation_2,
        nn.Linear(neurons_h3, 1)
    )
    
    _optimizer = torch.optim.Adam(
        model.parameters(),
        lr=adam_lr,
        betas=(adam_beta1, adam_beta2),
        eps=adam_epsilon
    )
    
    model.to(device)
        
    for epoch in range(num_epochs):
        # --- Training Phase ---
        model.train() # Set model to training mode (enables dropout, batchnorm updates)
        running_train_loss = 0.0
        for batch_idx, (features, targets) in enumerate(train_loader):
            # Move batch data to the target device (GPU or CPU)
            features, targets = features.to(device), targets.to(device)
            _optimizer.zero_grad()
            outputs = model(features)
            # Reshape the targets tensor to match the outputs shape ([batch_size, 1])
            targets_reshaped = targets.unsqueeze(1)
            loss = loss_func(outputs, targets_reshaped)
            loss.backward()
            _optimizer.step()
    
            running_train_loss += loss.item() * features.size(0)
    
        epoch_train_loss = running_train_loss / len(train_loader.dataset)
    
        # --- Validation Phase ---
        model.eval()
        running_val_loss = 0.0
        with torch.no_grad():
            for features, targets in val_loader:
                features, targets = features.to(device), targets.to(device)
                outputs = model(features)
                targets_reshaped = targets.unsqueeze(1)
                loss = loss_func(outputs, targets_reshaped)
                running_val_loss += loss.item() * features.size(0)
    
        epoch_val_loss = running_val_loss / len(val_loader.dataset)
        
        trial.report(epoch_val_loss, epoch) # Report intermediate value for pruning
        if trial.should_prune():
            print(f"Trial {trial.number} pruned at epoch {epoch+1}.")
            raise optuna.exceptions.TrialPruned()
        
        return epoch_val_loss

study = optuna.create_study(
    direction='minimize', 
    pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=150, interval_steps=10) 
    # n_startup_trials: Don't prune first 5 trials.
    # n_warmup_steps: Don't prune a trial before it has completed 50 epochs.
    # interval_steps: Check for pruning every 10 epochs after warmup.
)

# 2. Run the optimization.
#    Optuna will call your 'objective' function 'n_trials' times.
#    Each time, it passes a 'trial' object to your function.
study.optimize(objective, n_trials=4000)

# 3. Get the best results.
print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(study.get_trials(states=[optuna.trial.TrialState.PRUNED])))
print("  Number of complete trials: ", len(study.get_trials(states=[optuna.trial.TrialState.COMPLETE])))

print("\nBest trial:")
best_trial = study.best_trial

print("  Value (Min Validation Loss): ", best_trial.value)

print("  Best hyperparameters: ")
for key, value in best_trial.params.items():
    print(f"    {key}: {value}")

    # Print progress (e.g., every epoch or every few epochs)
#    print(f'Epoch {epoch+1}/{num_epochs} | Train Loss: {epoch_train_loss:.6f} | Val Loss: {epoch_val_loss:.6f}')

In [40]:
num_epochs = 500 # Hyperparameter: How many times to iterate over the dataset

best_validation_loss = float('inf')
best_model_path = "best_model_weights.pth"
patience_counter = 0
patience_epochs = 50

print("\nStarting Training...")
for epoch in range(num_epochs):
    # --- Training Phase ---
    model.train() # Set model to training mode (enables dropout, batchnorm updates)
    running_train_loss = 0.0
    for batch_idx, (features, targets) in enumerate(train_loader):
        # Move batch data to the target device (GPU or CPU)
        features, targets = features.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(features)
        # Reshape the targets tensor to match the outputs shape ([batch_size, 1])
        targets_reshaped = targets.unsqueeze(1)
        loss = loss_func(outputs, targets_reshaped)
        loss.backward()
        optimizer.step()

        running_train_loss += loss.item() * features.size(0)

    epoch_train_loss = running_train_loss / len(train_loader.dataset)

    # --- Validation Phase ---
    model.eval() # Set model to evaluation mode (disables dropout, batchnorm updates)
    running_val_loss = 0.0

    with torch.no_grad(): # No need to calculate gradients during validation
        for features, targets in val_loader:
            features, targets = features.to(device), targets.to(device)
            outputs = model(features)
            targets_reshaped = targets.unsqueeze(1)
            loss = loss_func(outputs, targets_reshaped)
            running_val_loss += loss.item() * features.size(0)

    epoch_val_loss = running_val_loss / len(val_loader.dataset)

    # Print progress (e.g., every epoch or every few epochs)
    print(f'Epoch {epoch+1}/{num_epochs} | Train Loss: {epoch_train_loss:.6f} | Val Loss: {epoch_val_loss:.6f}')
    if epoch_val_loss < best_validation_loss:
        print(f"Validation loss improved from {best_validation_loss:.6f} to {epoch_val_loss:.6f}. Saving model to {best_model_path}")
        best_validation_loss = epoch_val_loss
        torch.save(model.state_dict(), best_model_path) # Save the model's weights
        patience_counter = 0  # Reset patience since we found a better model
    else:
        patience_counter += 1
    
    if patience_counter >= patience_epochs:
        print(f"Early stopping triggered after {patience_epochs} epochs without improvement.")
        break



Starting Training...
Epoch 1/500 | Train Loss: 3194084.481544 | Val Loss: 31.439160
Validation loss improved from inf to 31.439160. Saving model to best_model_weights.pth
Epoch 2/500 | Train Loss: 5.767370 | Val Loss: 3.082345
Validation loss improved from 31.439160 to 3.082345. Saving model to best_model_weights.pth
Epoch 3/500 | Train Loss: 0.848507 | Val Loss: 0.130477
Validation loss improved from 3.082345 to 0.130477. Saving model to best_model_weights.pth
Epoch 4/500 | Train Loss: 0.108597 | Val Loss: 0.122169
Validation loss improved from 0.130477 to 0.122169. Saving model to best_model_weights.pth
Epoch 5/500 | Train Loss: 0.097866 | Val Loss: 0.121590
Validation loss improved from 0.122169 to 0.121590. Saving model to best_model_weights.pth
Epoch 6/500 | Train Loss: 0.097812 | Val Loss: 0.120879
Validation loss improved from 0.121590 to 0.120879. Saving model to best_model_weights.pth
Epoch 7/500 | Train Loss: 0.097853 | Val Loss: 0.121870
Epoch 8/500 | Train Loss: 0.097621 |

Model mathematically represented, using Adam optimizer, default hyperparameters:
$$
\begin{align}
% Input
\mathbf{a}^{(0)} &= \mathbf{x}, \quad \text{where } \mathbf{x} \in \mathbb{R}^{260} \\
% Layer 1
\mathbf{z}^{(1)} &= W^{(1)}\mathbf{a}^{(0)} + \mathbf{b}^{(1)}, \quad \text{where } W^{(1)} \in \mathbb{R}^{128 \times 260}, \mathbf{b}^{(1)} \in \mathbb{R}^{128} \\
\mathbf{a}^{(1)} &= \text{ReLU}(\mathbf{z}^{(1)}), \quad \mathbf{a}^{(1)} \in \mathbb{R}^{128} \\
% Layer 2
\mathbf{z}^{(2)} &= W^{(2)}\mathbf{a}^{(1)} + \mathbf{b}^{(2)}, \quad \text{where } W^{(2)} \in \mathbb{R}^{64 \times 128}, \mathbf{b}^{(2)} \in \mathbb{R}^{64} \\
\mathbf{a}^{(2)} &= \text{ReLU}(\mathbf{z}^{(2)}), \quad \mathbf{a}^{(2)} \in \mathbb{R}^{64} \\
% Layer 3
\mathbf{z}^{(3)} &= W^{(3)}\mathbf{a}^{(2)} + \mathbf{b}^{(3)}, \quad \text{where } W^{(3)} \in \mathbb{R}^{16 \times 64}, \mathbf{b}^{(3)} \in \mathbb{R}^{16} \\
\mathbf{a}^{(3)} &= \text{ReLU}(\mathbf{z}^{(3)}), \quad \mathbf{a}^{(3)} \in \mathbb{R}^{16} \\
% Layer 4 (Output)
\mathbf{z}^{(4)} &= W^{(4)}\mathbf{a}^{(3)} + b^{(4)}, \quad \text{where } W^{(4)} \in \mathbb{R}^{1 \times 16}, \mathbf{b}^{(4)} \in \mathbb{R} \\
\hat{y} &= \mathbf{z}^{(4)}, \quad \hat{y} \in \mathbb{R} \\
% Loss
\mathcal{L} &= (\hat{y} - y_{\text{target}})^2
\end{align}
$$

In [43]:
try:
    weights = torch.load(best_model_path, map_location=torch.device('cpu'))
    print("Successfully loaded entire model.")
    model.path = weights
    # You can now inspect the model structure
    print("\nModel structure:")
    print(model)

except Exception as e:
    print(f"Error loading .pth file: {e}")

Successfully loaded entire model.

Model structure:
OrderedDict({'0.weight': tensor([[-0.3538, -0.4171, -0.3354,  ...,  0.3727,  0.3174, -0.1427],
        [-0.3604, -0.3398, -0.3795,  ..., -0.3307,  0.3510, -0.3453],
        [-0.3268, -0.3873, -0.3551,  ...,  0.3550,  0.3401, -0.2410],
        ...,
        [-0.4087, -0.3104, -0.3129,  ..., -0.4217,  0.4192, -0.3423],
        [-0.0350, -0.0011, -0.0502,  ..., -0.0418, -0.0619, -0.0198],
        [-0.3049, -0.3309, -0.3674,  ...,  0.3650,  0.3439, -0.1259]]), '0.bias': tensor([-4.2232e-01, -3.2606e-01, -3.8081e-01, -3.9244e-01, -1.1622e-02,
        -5.4648e-02, -3.1089e-01, -5.6269e-03, -4.0108e-01, -3.2437e-01,
        -3.5242e-02, -3.5957e-01, -2.9641e-01, -5.5680e-02,  5.9336e-02,
        -4.1915e-01, -3.4381e-01,  3.4212e-02, -3.4749e-01, -3.5518e-01,
        -3.5186e-03, -2.4463e-02, -2.3829e-01, -3.7933e-01,  5.7697e-02,
        -2.9904e-01, -3.8016e-01, -2.4829e-03, -3.1815e-01,  5.8168e-02,
         1.5001e-01, -2.6798e-01, -3.801